In [1]:
import gc
import numpy as np
run = 0
np.random.seed(run)
import os, glob
import time
import pyarrow as pa
import pyarrow.parquet as pq
import torch
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import mplhep as hep
plt.style.use([hep.style.ROOT, hep.style.firamath])
#from skimage.transform import rescale
plt.rcParams["figure.figsize"] = (5,5)
from torch.utils.data import *
import pandas as pd
import pickle
print(torch.__version__)


1.12.1


In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True
print(device)

cuda:0


In [3]:
device = torch.device("cuda" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True
if use_cuda:
    device_ids = [0, 1, 2, 3]  # IDs of the 4 GPUs
else:
    device_ids = None 

In [4]:
print(device)

cuda


In [5]:
torch.cuda.device_count()



4

In [6]:
torch.cuda.get_device_name()

'NVIDIA A100-SXM4-40GB'

In [7]:
torch.cuda.memory_allocated()

0

In [8]:
import mass_trainer_aToTauTau_m1p2To17p2_v2_13ch_tets_only.torch_resnet_concat as networks

In [9]:
# def transform_y(y):
#     return y/m0_scale

# def inv_transform(y):
#     return y*m0_scale

# class ParquetDataset(Dataset):
#     def __init__(self, filename, label):
#         self.parquet = pq.ParquetFile(filename)
#         #self.cols = None # read all columns
#         #self.cols = ['X_jet.list.item.list.item.list.item','am','apt','iphi','ieta']
#         self.cols = ['X_jet.list.item.list.item.list.item','am','iphi','ieta']
#         self.label = label
#     def __getitem__(self, index):
#         data = self.parquet.read_row_group(index, columns=self.cols).to_pydict()
#         data['X_jet'] = np.float32(data['X_jet'][0])
#         data['X_jet'][0] = pt_scale   * data['X_jet'][0] #Track pT
#         data['X_jet'][1] = dz_scale   * data['X_jet'][1] #Track dZ
#         data['X_jet'][2] = dz_scale   * data['X_jet'][2] #Track d0
#         data['X_jet'][3] = ecal_scale * data['X_jet'][3] #ECAL
#         data['X_jet'][4] = hcal_scale * data['X_jet'][4] #HCAL
#         #data['X_jet'] = np.float32(data['X_jet'][0])/ecal_scale
#         data['am'] = transform_y(np.float32(data['am']))
#         #data['apt'] = np.float32(data['apt'])
#         data['iphi'] = np.float32(data['iphi'])/360.
#         data['ieta'] = np.float32(data['ieta'])/140.
#         data['label'] = self.label
#         # Preprocessing
#         #data_dict['X_jet'] = data_dict['X_jet'][:, 20:105, 20:105]
#         # High Value Suppressuib
#         data['X_jet'][1][data['X_jet'][1] < -20] = 0
#         data['X_jet'][1][data['X_jet'][1] >  20] = 0
#         data['X_jet'][2][data['X_jet'][2] < -10] = 0
#         data['X_jet'][2][data['X_jet'][2] >  10] = 0
#         # Zero-Suppression
#         data['X_jet'][0][data['X_jet'][0] < 1.e-3] = 0.
#         # data['X_jet'][1][data['X_jet'][1] < 1.e-4] = 0.
#         # data['X_jet'][2][data['X_jet'][2] < 1.e-4] = 0.
#         data['X_jet'][3][data['X_jet'][3] < 1.e-3] = 0.
#         data['X_jet'][4][data['X_jet'][4] < 1.e-3] = 0.

#         indices = [0,1,2,3,4,5,6,7,8,9,10,11,12]
#         newdata = [data['X_jet'][index,:,:] for index in indices]
#         data['X_jet'] = np.reshape(newdata, (len(indices),125,125))
#         return dict(data)
#     def __len__(self):
#         return self.parquet.num_row_groups
    
# def mae_loss_wgtd(pred, true, wgt=1.):
#     loss = wgt*(pred-true).abs().to(device)
#     return loss.mean()    

# def do_eval(resnet, val_loader, mae_best, epoch, sample):
#     torch.cuda.empty_cache()
#     global expt_name
#     loss_ = 0.
#     m_pred_, m_true_, mae_, mre_ = [], [], [], []
#     # iphi_, ieta_ = [], []
#     now = time.time()
#     ma_low = transform_y(3.6) # convert from GeV to network units
#     for i, data in enumerate(val_loader):
#         X, am = data['X_jet'].to(device), data['am'].to(device)
#         iphi, ieta = data['iphi'].to(device), data['ieta'].to(device)
#         logits = resnet([X, iphi, ieta])
#         loss_ += mae_loss_wgtd(logits, am).item()
#         logits, am = inv_transform(logits), inv_transform(am)
#         mae = (logits-am).abs()
#         mre = (((logits-am).abs())/am)

#         if i % 100 == 0:

#             print('Validation (%d/%d): Train loss:%f, mae:%f, mre:%f'%(i, len(val_loader), loss_/(i+1), mae.mean().item(), mre.mean().item() ))
#         # Store batch metrics:
#         m_pred_.append(logits.tolist())
#         m_true_.append(am.tolist())
#         mae_.append(mae.tolist())
#         mre_.append(mre.tolist())
        
#         del logits
#         gc.collect()
#     now = time.time() - now
#     m_true_ = np.concatenate(m_true_)
#     m_pred_ = np.concatenate(m_pred_)
#     mae_    = np.concatenate(mae_)
#     mre_    = np.concatenate(mre_)
#     score_str = 'epoch%d_%s_mae%.4f'%(epoch, sample, np.mean(mae_))
#     lr_scheduler.step(loss_/len(val_loader))
#     print(optimizer.param_groups[0]['lr'])
    
#     gc.collect()

#     mae_retun = np.mean(mae_)
#     return mae_retun


In [10]:
def transform_y(y):
    return y/m0_scale

def inv_transform(y):
    return y*m0_scale

class ParquetDataset(Dataset):
    def __init__(self, filename, label):
        self.parquet = pq.ParquetFile(filename)
        #self.cols = None # read all columns
        #self.cols = ['X_jet.list.item.list.item.list.item','am','apt','iphi','ieta']
        self.cols = ['X_jet.list.item.list.item.list.item','am','iphi','ieta']
        self.label = label
    def __getitem__(self, index):
        data = self.parquet.read_row_group(index, columns=self.cols).to_pydict()
        data['X_jet'] = torch.tensor(data['X_jet'][0], dtype=torch.float32)
        data['am'] = torch.tensor(transform_y(np.float32(data['am'])), dtype=torch.float32)
        data['iphi'] = torch.tensor(np.float32(data['iphi'])/360., dtype=torch.float32)
        data['ieta'] = torch.tensor(np.float32(data['ieta'])/140, dtype=torch.float32)
        return data
    def __len__(self):
        return self.parquet.num_row_groups
    def __len__(self):
        return self.parquet.num_row_groups
    
def mae_loss_wgtd(pred, true, wgt=1.):
    loss = wgt*(pred-true).abs().to(device)
    return loss.mean()    




In [15]:
lr_init = 5.e-4
lr_factor = 0.2
new_lr = 0
patience = 2
resblocks = 3
epochs = 2
load_epoch = 0

hcal_scale  = 1
ecal_scale  = 0.2
pt_scale    = 0.02
dz_scale    = 10
m0_scale    = 14
m0_min      = 3.6
m0_min_un   = 1.2
m0_max      = 14
mtrue_bins = np.arange(m0_min, m0_max, .4)
mae_min  =0
mae_max  = 10.5
mae_bins = np.arange(mae_min, mae_max, .5)
mre_bins = np.arange(0, 1.05, .05)
mass_bins = np.arange(3600,14000,400)/1000.
BATCH_SIZE = 256

n_train = ( 1000 // BATCH_SIZE ) * BATCH_SIZE
n_val = (500// BATCH_SIZE ) * BATCH_SIZE
# n_train = ( 5296937 // BATCH_SIZE ) * BATCH_SIZE
# n_val = ( 597421// BATCH_SIZE ) * BATCH_SIZE

channel_list = ["Tracks_pt", "Tracks_dZSig", "Tracks_d0Sig", "ECAL_energy",
"HBHE_energy", "Pix_1", "Pix_2", "Pix_3", "Pix_4", "Tib_1", "Tib_2",
"Tib_3", "Tib_4", "Tob_1", "Tob_2", "Tob_3", "Tob_4", "Tob_5",
"Tob_6", "Tid_1", "Tec_1", "Tec_2", "Tec_3"]

indices = [0,1,2,3,4,5,6,7,8,9,10,11,12]  # channel selected for training change it to class defination too
channels_used = [channel_list[ch] for ch in indices]
layers_names = '_'.join(channels_used)

resnet = networks.ResNet(len(indices), resblocks, [8, 16, 32, 64])

In [16]:
#!ls /pscratch/sd/b/bbbam/IMG_aToTauTau_Hadronic_tauDR0p4_m1p2To17p2_dataset_2_unbaised_v2_train/

In [17]:
train_decays = glob.glob('/pscratch/sd/b/bbbam/IMG_aToTauTau_Hadronic_tauDR0p4_m1p2To17p2_dataset_2_unbaised_v2_train/IMG_aToTauTau_Hadronic_tauDR0p4_m1p2To3p6_dataset_2_unbaised_unphysical_0007_train.parquet')

dset_train = ConcatDataset([ParquetDataset('%s'%d,i) for i,d in enumerate(train_decays)])


idxs_train_t = np.random.permutation(len(dset_train))

idxs_train = np.random.permutation(n_train)


# Train dataset
train_sampler = RandomSampler(dset_train, replacement=True, num_samples=n_train)
train_loader  = DataLoader(dataset=dset_train, batch_size=BATCH_SIZE, num_workers=1, pin_memory=True, sampler=train_sampler)





In [18]:
resnet.to(device)
optimizer = optim.Adam(resnet.parameters(), lr=lr_init)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=lr_factor, patience=patience)

In [19]:
def zs(data):
    data[:, 0, :, :] = pt_scale * data[:, 0, :, :]   #Track pT
    data[:, 1, :, :] = dz_scale   * data[:, 1, :, :] #Track dZ
    data[:, 2, :, :] = dz_scale   * data[:, 2, :, :] #Track d0
    data[:, 3, :, :] = ecal_scale * data[:, 3, :, :] #ECAL
    data[:, 4, :, :] = hcal_scale * data[:, 4, :, :] #HCAL
    # Preprocessing
    #data_dict['X_jet'] = data_dict['X_jet'][:, 20:105, 20:105]
    # High Value Suppressuib
    data[:, 1, :, :][data[:, 1, :, :] < -20] = 0
    data[:, 1, :, :][data[:, 1, :, :] >  20] = 0
    data[:, 2, :, :][data[:, 2, :, :] < -10] = 0
    data[:, 2, :, :][data[:, 2, :, :] >  10] = 0
    # Zero-Suppression
    data[:, 0, :, :][data[:, 0, :, :] < 1.e-3] = 0.
    # data['X_jet'][1][data['X_jet'][1] < 1.e-4] = 0.
    # data['X_jet'][2][data['X_jet'][2] < 1.e-4] = 0.
    data[:, 3, :, :][data[:, 3, :, :] < 1.e-3] = 0.
    data[:, 4, :, :][data[:, 4, :, :] < 1.e-3] = 0.

    data = torch.index_select(data, 1, torch.tensor(indices).to(device))
    return data

In [21]:
run_logger = False
print_step = 10
mae_best = 1
for e in range(1):
    epoch = e+1+load_epoch
    epoch_wgt = 0.
    n_trained = 0
    resnet.train()
    now = time.time()
    for i, data in enumerate(train_loader):
        # X =  data['X_jet'].clone().detach().to(device).requires_grad_(True)
        X =  data['X_jet'].to(device)
        # print("X device----  ",X.device)
        # print(" X shape---  ",X.shape)
        # print(" X is_leaf---  ",X.is_leaf)
        # print(" X[:,0,:,:] shape  ",X[:,0,:,:].shape)
        # print(" X[:,0,:,:] device  ",X[:,0,:,:].device)
        with torch.no_grad():
             # X[:, 0, :, :] = pt_scale * X[:, 0, :, :]   #Track pT
             # X[:, 1, :, :] = dz_scale   * X[:, 1, :, :] #Track dZ
             # X[:, 2, :, :] = dz_scale   * X[:, 2, :, :] #Track d0
             # X[:, 3, :, :] = ecal_scale * X[:, 3, :, :] #ECAL
             # X[:, 4, :, :] = hcal_scale * X[:, 4, :, :] #HCAL
             # # Preprocessing
             # #X_dict['X_jet'] = X_dict['X_jet'][:, 20:105, 20:105]
             # # High Value Suppressuib
             # X[:, 1, :, :][X[:, 1, :, :] < -20] = 0
             # X[:, 1, :, :][X[:, 1, :, :] >  20] = 0
             # X[:, 2, :, :][X[:, 2, :, :] < -10] = 0
             # X[:, 2, :, :][X[:, 2, :, :] >  10] = 0
             # # Zero-Suppression
             # X[:, 0, :, :][X[:, 0, :, :] < 1.e-3] = 0.
             # # X['X_jet'][1][X['X_jet'][1] < 1.e-4] = 0.
             # # X['X_jet'][2][X['X_jet'][2] < 1.e-4] = 0.
             # X[:, 3, :, :][X[:, 3, :, :] < 1.e-3] = 0.
             # X[:, 4, :, :][X[:, 4, :, :] < 1.e-3] = 0.
             # print("---------------------------------")
             # print(" X[:,0,:,:] shape  ",X[:,0,:,:].shape)
             # print(" X[:,0,:,:] device  ",X[:,0,:,:].device)
             # print(" X[:,1,:,:] device  ",X[:,1,:,:].device)
             # print(" X[:,2,:,:] device  ",X[:,2,:,:].device)
             # print(" X[:,3,:,:] device  ",X[:,3,:,:].device)
             # print(" X[:,4,:,:] device  ",X[:,4,:,:].device)
             # print(" X[:,5,:,:] device  ",X[:,5,:,:].device)
             # print(" X[:,6,:,:] device  ",X[:,6,:,:].device)
             # print(" X[:,7,:,:] device  ",X[:,7,:,:].device)
             # print(" X[:,8,:,:] device  ",X[:,8,:,:].device)
             # print(" X[:,9,:,:] device  ",X[:,9,:,:].device)
             # print(" X[:,10,:,:] device  ",X[:,10,:,:].device)
             # print(" X[:,11,:,:] device  ",X[:,11,:,:].device)
             # print(" X[:,12,:,:] device  ",X[:,12,:,:].device)
             # X = torch.index_select(X, 1, torch.tensor(indices).to(device))
             # print(" X1 device  ",X.device)
             # print(" X1 shape  ",X.shape)
             X= zs(X)
        am = data['am'].to(device)
        print("am device  ",am.device)
        print(" am shape  ",am.shape)
        iphi = data['iphi'].to(device)
        print("iphi device  ",iphi.device)
        print(" iphi shape  ",iphi.shape)
        ieta = data['ieta'].to(device)
        print("ieta device  ",ieta.device)
        print(" ieta shape  ",ieta.shape)
        optimizer.zero_grad()
        logits = resnet([X, iphi, ieta])
        loss = mae_loss_wgtd(logits, am)
        loss.backward()
        optimizer.step()
        epoch_wgt += len(am)
        n_trained += 1
        if i % print_step == 0:
            logits, am = inv_transform(logits), inv_transform(am)
            mae =  (logits-am).abs().mean()
            mre = (((logits-am).abs())/am).mean()
            print('%d: (%d/%d) m_pred: %s...'%(epoch, i, len(train_loader), str(np.squeeze(logits.tolist()[:5]))))
            print('%d: (%d/%d) m_true: %s...'%(epoch, i, len(train_loader), str(np.squeeze(am.tolist()[:5]))))
            print('%d: (%d/%d) Train loss:%f, mae:%f, mre:%f'%(epoch, i, len(train_loader), loss.item(), mae.item(), mre.item() ))

    now = time.time() - now
    logits, am = inv_transform(logits), inv_transform(am)
    mae = (logits-am).abs().mean()
    mre = ((logits-am).abs()/am).mean()
    print('%d: Train time:%.2fs in %d steps for N:%d, wgt: %.f'%(epoch, now, len(train_loader), n_trained, epoch_wgt))
    print('%d: Train loss:%f, mae:%f, mre:%f'%(epoch, loss.item(), mae.item(), mre.item() ))

#     # Run Validation
#     resnet.eval()
#     _ = do_eval(resnet, val_loader, mae_best, epoch, 'val_pseudoscalar')
    

am device   cuda:0
 am shape   torch.Size([256, 1])
iphi device   cuda:0
 iphi shape   torch.Size([256, 1])
ieta device   cuda:0
 ieta shape   torch.Size([256, 1])
1: (0/3) m_pred: [2.71225405 2.63376641 2.60041237 2.70812058 2.60632753]...
1: (0/3) m_true: [1.88051677 3.24924231 3.15504909 3.06960058 3.47096062]...
1: (0/3) Train loss:0.045145, mae:0.632029, mre:0.329382
am device   cuda:0
 am shape   torch.Size([256, 1])
iphi device   cuda:0
 iphi shape   torch.Size([256, 1])
ieta device   cuda:0
 ieta shape   torch.Size([256, 1])
am device   cuda:0
 am shape   torch.Size([256, 1])
iphi device   cuda:0
 iphi shape   torch.Size([256, 1])
ieta device   cuda:0
 ieta shape   torch.Size([256, 1])
1: Train time:43.48s in 3 steps for N:3, wgt: 768
1: Train loss:0.062311, mae:0.872360, mre:0.503203
